In [ ]:
import asyncio
import os
from io import BytesIO
from itertools import batched

import httpx
import pandas as pd

In [3]:
def read_msci_excel(file: bytes):
    return pd.read_excel(
        (BytesIO(file)),
        engine="calamine",
        skiprows=6,
        skipfooter=19,
        parse_dates=True,
        date_format="%b %d, %Y",
        thousands=",",
        index_col="Date",
    )

In [4]:
indexes = (
    ("WORLD/STANDARD/BLEND", "WORLD", "990100"),
    ("WORLD/STANDARD/VALUE", "WORLD VALUE", "105868"),
    ("WORLD/STANDARD/GROWTH", "WORLD GROWTH", "105867"),
    ("WORLD/SMALL/BLEND", "WORLD SMALL CAP", "106230"),
    ("WORLD/SMALL/VALUE", "WORLD SMALL VALUE", "655131"),
    ("WORLD/SMALL/GROWTH", "WORLD SMALL GROWTH", "655236"),
    ("WORLD/SMID/BLEND", "WORLD SMID CAP", "661668"),
    ("WORLD/SMID/VALUE", "WORLD SMID VALUE", "661773"),
    ("WORLD/SMID/GROWTH", "WORLD SMID GROWTH", "661878"),
    ("WORLD/MID/BLEND", "WORLD MID CAP", "652555"),
    ("WORLD/MID/VALUE", "WORLD MID VALUE", "652660"),
    ("WORLD/MID/GROWTH", "WORLD MID GROWTH", "652758"),
    ("WORLD/LARGE/BLEND", "WORLD LARGE CAP", "650038"),
    ("WORLD/LARGE/VALUE", "WORLD LARGE VALUE", "650143"),
    ("WORLD/LARGE/GROWTH", "WORLD LARGE GROWTH", "650248"),
    ("WORLD/IMI/BLEND", "WORLD IMI", "664185"),
    ("WORLD/IMI/VALUE", "WORLD IMI VALUE", "664283"),
    ("WORLD/IMI/GROWTH", "WORLD IMI GROWTH", "664395"),
    ("ACWI/STANDARD/BLEND", "ACWI", "892400"),
    ("ACWI/STANDARD/VALUE", "ACWI VALUE", "106039"),
    ("ACWI/STANDARD/GROWTH", "ACWI GROWTH", "106038"),
    ("ACWI/SMALL/BLEND", "ACWI SMALL CAP", "655045"),
    ("ACWI/SMALL/VALUE", "ACWI SMALL VALUE", "655150"),
    ("ACWI/SMALL/GROWTH", "ACWI SMALL GROWTH", "655255"),
    ("ACWI/SMID/BLEND", "ACWI SMID CAP", "661687"),
    ("ACWI/SMID/VALUE", "ACWI SMID VALUE", "661792"),
    ("ACWI/SMID/GROWTH", "ACWI SMID GROWTH", "661897"),
    ("ACWI/MID/BLEND", "ACWI MID CAP", "652574"),
    ("ACWI/MID/VALUE", "ACWI MID VALUE", "652679"),
    ("ACWI/MID/GROWTH", "ACWI MID GROWTH", "652784"),
    ("ACWI/LARGE/BLEND", "ACWI LARGE CAP", "650057"),
    ("ACWI/LARGE/VALUE", "ACWI LARGE VALUE", "650162"),
    ("ACWI/LARGE/GROWTH", "ACWI LARGE GROWTH", "650267"),
    ("ACWI/IMI/BLEND", "ACWI IMI", "664204"),
    ("ACWI/IMI/VALUE", "ACWI IMI VALUE", "664309"),
    ("ACWI/IMI/GROWTH", "ACWI IMI GROWTH", "664414"),
    ("EM (EMERGING MARKETS)/STANDARD/BLEND", "EM (EMERGING MARKETS)", "891800"),
    ("EM (EMERGING MARKETS)/STANDARD/VALUE", "EM (EMERGING MARKETS) VALUE", "106063"),
    ("EM (EMERGING MARKETS)/STANDARD/GROWTH", "EM (EMERGING MARKETS) GROWTH", "106062"),
    ("EM (EMERGING MARKETS)/SMALL/BLEND", "EM (EMERGING MARKETS) SMALL CAP", "655061"),
    (
        "EM (EMERGING MARKETS)/SMALL/VALUE",
        "EM (EMERGING MARKETS) SMALL VALUE",
        "655166",
    ),
    (
        "EM (EMERGING MARKETS)/SMALL/GROWTH",
        "EM (EMERGING MARKETS) SMALL GROWTH",
        "655271",
    ),
    ("EM (EMERGING MARKETS)/SMID/BLEND", "EM (EMERGING MARKETS) SMID CAP", "661703"),
    ("EM (EMERGING MARKETS)/SMID/VALUE", "EM (EMERGING MARKETS) SMID VALUE", "661808"),
    (
        "EM (EMERGING MARKETS)/SMID/GROWTH",
        "EM (EMERGING MARKETS) SMID GROWTH",
        "661913",
    ),
    ("EM (EMERGING MARKETS)/MID/BLEND", "EM (EMERGING MARKETS) MID CAP", "652590"),
    ("EM (EMERGING MARKETS)/MID/VALUE", "EM (EMERGING MARKETS) MID VALUE", "652695"),
    ("EM (EMERGING MARKETS)/MID/GROWTH", "EM (EMERGING MARKETS) MID GROWTH", "652800"),
    ("EM (EMERGING MARKETS)/LARGE/BLEND", "EM (EMERGING MARKETS) LARGE CAP", "650073"),
    (
        "EM (EMERGING MARKETS)/LARGE/VALUE",
        "EM (EMERGING MARKETS) LARGE VALUE",
        "650178",
    ),
    (
        "EM (EMERGING MARKETS)/LARGE/GROWTH",
        "EM (EMERGING MARKETS) LARGE GROWTH",
        "650283",
    ),
    ("EM (EMERGING MARKETS)/IMI/BLEND", "EM (EMERGING MARKETS) IMI", "664220"),
    ("EM (EMERGING MARKETS)/IMI/VALUE", "EM (EMERGING MARKETS) IMI VALUE", "664325"),
    ("EM (EMERGING MARKETS)/IMI/GROWTH", "EM (EMERGING MARKETS) IMI GROWTH", "664430"),
    (
        "KOKUSAI INDEX (WORLD ex JP)/STANDARD/BLEND",
        "KOKUSAI INDEX (WORLD ex JP)",
        "991200",
    ),
    (
        "KOKUSAI INDEX (WORLD ex JP)/STANDARD/VALUE",
        "KOKUSAI INDEX (WORLD ex JP) VALUE",
        "105850",
    ),
    (
        "KOKUSAI INDEX (WORLD ex JP)/STANDARD/GROWTH",
        "KOKUSAI INDEX (WORLD ex JP) GROWTH",
        "105849",
    ),
    (
        "KOKUSAI INDEX (WORLD ex JP)/SMALL/BLEND",
        "KOKUSAI INDEX (WORLD ex JP) SMALL CAP",
        "106239",
    ),
    (
        "KOKUSAI INDEX (WORLD ex JP)/SMALL/VALUE",
        "KOKUSAI INDEX (WORLD ex JP) SMALL VALUE",
        "655118",
    ),
    (
        "KOKUSAI INDEX (WORLD ex JP)/SMALL/GROWTH",
        "KOKUSAI INDEX (WORLD ex JP) SMALL GROWTH",
        "655223",
    ),
    (
        "KOKUSAI INDEX (WORLD ex JP)/SMID/BLEND",
        "KOKUSAI INDEX (WORLD ex JP) SMID CAP",
        "661655",
    ),
    (
        "KOKUSAI INDEX (WORLD ex JP)/SMID/VALUE",
        "KOKUSAI INDEX (WORLD ex JP) SMID VALUE",
        "661760",
    ),
    (
        "KOKUSAI INDEX (WORLD ex JP)/SMID/GROWTH",
        "KOKUSAI INDEX (WORLD ex JP) SMID GROWTH",
        "661865",
    ),
    (
        "KOKUSAI INDEX (WORLD ex JP)/MID/BLEND",
        "KOKUSAI INDEX (WORLD ex JP) MID CAP",
        "652542",
    ),
    (
        "KOKUSAI INDEX (WORLD ex JP)/MID/VALUE",
        "KOKUSAI INDEX (WORLD ex JP) MID VALUE",
        "652647",
    ),
    (
        "KOKUSAI INDEX (WORLD ex JP)/MID/GROWTH",
        "KOKUSAI INDEX (WORLD ex JP) MID GROWTH",
        "652752",
    ),
    (
        "KOKUSAI INDEX (WORLD ex JP)/LARGE/BLEND",
        "KOKUSAI INDEX (WORLD ex JP) LARGE CAP",
        "650025",
    ),
    (
        "KOKUSAI INDEX (WORLD ex JP)/LARGE/VALUE",
        "KOKUSAI INDEX (WORLD ex JP) LARGE VALUE",
        "650130",
    ),
    (
        "KOKUSAI INDEX (WORLD ex JP)/LARGE/GROWTH",
        "KOKUSAI INDEX (WORLD ex JP) LARGE GROWTH",
        "650235",
    ),
    (
        "KOKUSAI INDEX (WORLD ex JP)/IMI/BLEND",
        "KOKUSAI INDEX (WORLD ex JP) IMI",
        "664172",
    ),
    (
        "KOKUSAI INDEX (WORLD ex JP)/IMI/VALUE",
        "KOKUSAI INDEX (WORLD ex JP) IMI VALUE",
        "664277",
    ),
    (
        "KOKUSAI INDEX (WORLD ex JP)/IMI/GROWTH",
        "KOKUSAI INDEX (WORLD ex JP) IMI GROWTH",
        "664382",
    ),
    ("WORLD ex USA/STANDARD/BLEND", "WORLD ex USA", "991000"),
    ("WORLD ex USA/STANDARD/VALUE", "WORLD ex USA VALUE", "105874"),
    ("WORLD ex USA/STANDARD/GROWTH", "WORLD ex USA GROWTH", "105873"),
    ("WORLD ex USA/SMALL/BLEND", "WORLD ex USA SMALL CAP", "106238"),
    ("WORLD ex USA/SMALL/VALUE", "WORLD ex USA SMALL VALUE", "655138"),
    ("WORLD ex USA/SMALL/GROWTH", "WORLD ex USA SMALL GROWTH", "655243"),
    ("WORLD ex USA/SMID/BLEND", "WORLD ex USA SMID CAP", "661675"),
    ("WORLD ex USA/SMID/VALUE", "WORLD ex USA SMID VALUE", "661780"),
    ("WORLD ex USA/SMID/GROWTH", "WORLD ex USA SMID GROWTH", "661885"),
    ("WORLD ex USA/MID/BLEND", "WORLD ex USA MID CAP", "652562"),
    ("WORLD ex USA/MID/VALUE", "WORLD ex USA MID VALUE", "652667"),
    ("WORLD ex USA/MID/GROWTH", "WORLD ex USA MID GROWTH", "652772"),
    ("WORLD ex USA/LARGE/BLEND", "WORLD ex USA LARGE CAP", "650045"),
    ("WORLD ex USA/LARGE/VALUE", "WORLD ex USA LARGE VALUE", "650150"),
    ("WORLD ex USA/LARGE/GROWTH", "WORLD ex USA LARGE GROWTH", "650255"),
    ("WORLD ex USA/IMI/BLEND", "WORLD ex USA IMI", "664192"),
    ("WORLD ex USA/IMI/VALUE", "WORLD ex USA IMI VALUE", "664297"),
    ("WORLD ex USA/IMI/GROWTH", "WORLD ex USA IMI GROWTH", "664402"),
    ("EUROPE/STANDARD/BLEND", "EUROPE", "990500"),
    ("EUROPE/STANDARD/VALUE", "EUROPE VALUE", "105844"),
    ("EUROPE/STANDARD/GROWTH", "EUROPE GROWTH", "105843"),
    ("EUROPE/SMALL/BLEND", "EUROPE SMALL CAP", "106233"),
    ("EUROPE/SMALL/VALUE", "EUROPE SMALL VALUE", "655103"),
    ("EUROPE/SMALL/GROWTH", "EUROPE SMALL GROWTH", "655208"),
    ("EUROPE/SMID/BLEND", "EUROPE SMID CAP", "661640"),
    ("EUROPE/SMID/VALUE", "EUROPE SMID VALUE", "661745"),
    ("EUROPE/SMID/GROWTH", "EUROPE SMID GROWTH", "661850"),
    ("EUROPE/MID/BLEND", "EUROPE MID CAP", "652527"),
    ("EUROPE/MID/VALUE", "EUROPE MID VALUE", "652632"),
    ("EUROPE/MID/GROWTH", "EUROPE MID GROWTH", "652737"),
    ("EUROPE/LARGE/BLEND", "EUROPE LARGE CAP", "650010"),
    ("EUROPE/LARGE/VALUE", "EUROPE LARGE VALUE", "650115"),
    ("EUROPE/LARGE/GROWTH", "EUROPE LARGE GROWTH", "650220"),
    ("EUROPE/IMI/BLEND", "EUROPE IMI", "664157"),
    ("EUROPE/IMI/VALUE", "EUROPE IMI VALUE", "664262"),
    ("EUROPE/IMI/GROWTH", "EUROPE IMI GROWTH", "664367"),
    ("AUSTRIA/STANDARD/BLEND", "AUSTRIA", "904000"),
    ("AUSTRIA/STANDARD/VALUE", "AUSTRIA VALUE", "105774"),
    ("AUSTRIA/STANDARD/GROWTH", "AUSTRIA GROWTH", "105773"),
    ("AUSTRIA/SMALL/BLEND", "AUSTRIA SMALL CAP", "106208"),
    ("AUSTRIA/SMALL/VALUE", "AUSTRIA SMALL VALUE", "655094"),
    ("AUSTRIA/SMALL/GROWTH", "AUSTRIA SMALL GROWTH", "655199"),
    ("AUSTRIA/SMID/BLEND", "AUSTRIA SMID CAP", "661631"),
    ("AUSTRIA/SMID/VALUE", "AUSTRIA SMID VALUE", "661736"),
    ("AUSTRIA/SMID/GROWTH", "AUSTRIA SMID GROWTH", "661841"),
    ("AUSTRIA/MID/BLEND", "AUSTRIA MID CAP", "652518"),
    ("AUSTRIA/MID/VALUE", "AUSTRIA MID VALUE", "652623"),
    ("AUSTRIA/LARGE/BLEND", "AUSTRIA LARGE CAP", "650001"),
    ("AUSTRIA/LARGE/GROWTH", "AUSTRIA LARGE GROWTH", "650211"),
    ("AUSTRIA/IMI/BLEND", "AUSTRIA IMI", "664148"),
    ("AUSTRIA/IMI/VALUE", "AUSTRIA IMI VALUE", "664253"),
    ("AUSTRIA/IMI/GROWTH", "AUSTRIA IMI GROWTH", "664358"),
    ("BELGIUM/STANDARD/BLEND", "BELGIUM", "905600"),
    ("BELGIUM/STANDARD/VALUE", "BELGIUM VALUE", "105776"),
    ("BELGIUM/STANDARD/GROWTH", "BELGIUM GROWTH", "105775"),
    ("BELGIUM/SMALL/BLEND", "BELGIUM SMALL CAP", "106209"),
    ("BELGIUM/SMALL/VALUE", "BELGIUM SMALL VALUE", "655095"),
    ("BELGIUM/SMALL/GROWTH", "BELGIUM SMALL GROWTH", "655200"),
    ("BELGIUM/SMID/BLEND", "BELGIUM SMID CAP", "661632"),
    ("BELGIUM/SMID/VALUE", "BELGIUM SMID VALUE", "661737"),
    ("BELGIUM/SMID/GROWTH", "BELGIUM SMID GROWTH", "661842"),
    ("BELGIUM/MID/BLEND", "BELGIUM MID CAP", "652519"),
    ("BELGIUM/MID/VALUE", "BELGIUM MID VALUE", "652624"),
    ("BELGIUM/MID/GROWTH", "BELGIUM MID GROWTH", "652729"),
    ("BELGIUM/LARGE/BLEND", "BELGIUM LARGE CAP", "650002"),
    ("BELGIUM/LARGE/VALUE", "BELGIUM LARGE VALUE", "650107"),
    ("BELGIUM/LARGE/GROWTH", "BELGIUM LARGE GROWTH", "650212"),
    ("BELGIUM/IMI/BLEND", "BELGIUM IMI", "664149"),
    ("BELGIUM/IMI/VALUE", "BELGIUM IMI VALUE", "664254"),
    ("BELGIUM/IMI/GROWTH", "BELGIUM IMI GROWTH", "664359"),
    ("DENMARK/STANDARD/BLEND", "DENMARK", "920800"),
    ("DENMARK/STANDARD/VALUE", "DENMARK VALUE", "105780"),
    ("DENMARK/STANDARD/GROWTH", "DENMARK GROWTH", "105779"),
    ("DENMARK/SMALL/BLEND", "DENMARK SMALL CAP", "106211"),
    ("DENMARK/SMALL/VALUE", "DENMARK SMALL VALUE", "655097"),
    ("DENMARK/SMALL/GROWTH", "DENMARK SMALL GROWTH", "655202"),
    ("DENMARK/SMID/BLEND", "DENMARK SMID CAP", "661634"),
    ("DENMARK/SMID/VALUE", "DENMARK SMID VALUE", "661739"),
    ("DENMARK/SMID/GROWTH", "DENMARK SMID GROWTH", "661844"),
    ("DENMARK/MID/BLEND", "DENMARK MID CAP", "652521"),
    ("DENMARK/MID/VALUE", "DENMARK MID VALUE", "652626"),
    ("DENMARK/MID/GROWTH", "DENMARK MID GROWTH", "652731"),
    ("DENMARK/LARGE/BLEND", "DENMARK LARGE CAP", "650004"),
    ("DENMARK/LARGE/VALUE", "DENMARK LARGE VALUE", "650109"),
    ("DENMARK/LARGE/GROWTH", "DENMARK LARGE GROWTH", "650214"),
    ("DENMARK/IMI/BLEND", "DENMARK IMI", "664151"),
    ("DENMARK/IMI/VALUE", "DENMARK IMI VALUE", "664256"),
    ("DENMARK/IMI/GROWTH", "DENMARK IMI GROWTH", "664361"),
    ("FINLAND/STANDARD/BLEND", "FINLAND", "924600"),
    ("FINLAND/STANDARD/VALUE", "FINLAND VALUE", "105784"),
    ("FINLAND/STANDARD/GROWTH", "FINLAND GROWTH", "105783"),
    ("FINLAND/SMALL/BLEND", "FINLAND SMALL CAP", "106212"),
    ("FINLAND/SMALL/VALUE", "FINLAND SMALL VALUE", "655110"),
    ("FINLAND/SMALL/GROWTH", "FINLAND SMALL GROWTH", "655215"),
    ("FINLAND/SMID/BLEND", "FINLAND SMID CAP", "661647"),
    ("FINLAND/SMID/VALUE", "FINLAND SMID VALUE", "661752"),
    ("FINLAND/SMID/GROWTH", "FINLAND SMID GROWTH", "661857"),
    ("FINLAND/MID/BLEND", "FINLAND MID CAP", "652534"),
    ("FINLAND/MID/VALUE", "FINLAND MID VALUE", "652639"),
    ("FINLAND/MID/GROWTH", "FINLAND MID GROWTH", "652744"),
    ("FINLAND/LARGE/BLEND", "FINLAND LARGE CAP", "650017"),
    ("FINLAND/LARGE/VALUE", "FINLAND LARGE VALUE", "650122"),
    ("FINLAND/LARGE/GROWTH", "FINLAND LARGE GROWTH", "650227"),
    ("FINLAND/IMI/BLEND", "FINLAND IMI", "664164"),
    ("FINLAND/IMI/VALUE", "FINLAND IMI VALUE", "664269"),
    ("FINLAND/IMI/GROWTH", "FINLAND IMI GROWTH", "664374"),
    ("FRANCE/STANDARD/BLEND", "FRANCE", "925000"),
    ("FRANCE/STANDARD/VALUE", "FRANCE VALUE", "105786"),
    ("FRANCE/STANDARD/GROWTH", "FRANCE GROWTH", "105785"),
    ("FRANCE/SMALL/BLEND", "FRANCE SMALL CAP", "106213"),
    ("FRANCE/SMALL/VALUE", "FRANCE SMALL VALUE", "655111"),
    ("FRANCE/SMALL/GROWTH", "FRANCE SMALL GROWTH", "655216"),
    ("FRANCE/SMID/BLEND", "FRANCE SMID CAP", "661648"),
    ("FRANCE/SMID/VALUE", "FRANCE SMID VALUE", "661753"),
    ("FRANCE/SMID/GROWTH", "FRANCE SMID GROWTH", "661858"),
    ("FRANCE/MID/BLEND", "FRANCE MID CAP", "652535"),
    ("FRANCE/MID/VALUE", "FRANCE MID VALUE", "652640"),
    ("FRANCE/MID/GROWTH", "FRANCE MID GROWTH", "652745"),
    ("FRANCE/LARGE/BLEND", "FRANCE LARGE CAP", "650018"),
    ("FRANCE/LARGE/VALUE", "FRANCE LARGE VALUE", "650123"),
    ("FRANCE/LARGE/GROWTH", "FRANCE LARGE GROWTH", "650228"),
    ("FRANCE/IMI/BLEND", "FRANCE IMI", "664165"),
    ("FRANCE/IMI/VALUE", "FRANCE IMI VALUE", "664270"),
    ("FRANCE/IMI/GROWTH", "FRANCE IMI GROWTH", "664375"),
    ("GERMANY/STANDARD/BLEND", "GERMANY", "928000"),
    ("GERMANY/STANDARD/VALUE", "GERMANY VALUE", "105788"),
    ("GERMANY/STANDARD/GROWTH", "GERMANY GROWTH", "105787"),
    ("GERMANY/SMALL/BLEND", "GERMANY SMALL CAP", "106214"),
    ("GERMANY/SMALL/VALUE", "GERMANY SMALL VALUE", "655112"),
    ("GERMANY/SMALL/GROWTH", "GERMANY SMALL GROWTH", "655217"),
    ("GERMANY/SMID/BLEND", "GERMANY SMID CAP", "661649"),
    ("GERMANY/SMID/VALUE", "GERMANY SMID VALUE", "661754"),
    ("GERMANY/SMID/GROWTH", "GERMANY SMID GROWTH", "661859"),
    ("GERMANY/MID/BLEND", "GERMANY MID CAP", "652536"),
    ("GERMANY/MID/VALUE", "GERMANY MID VALUE", "652641"),
    ("GERMANY/MID/GROWTH", "GERMANY MID GROWTH", "652746"),
    ("GERMANY/LARGE/BLEND", "GERMANY LARGE CAP", "650019"),
    ("GERMANY/LARGE/VALUE", "GERMANY LARGE VALUE", "650124"),
    ("GERMANY/LARGE/GROWTH", "GERMANY LARGE GROWTH", "650229"),
    ("GERMANY/IMI/BLEND", "GERMANY IMI", "664166"),
    ("GERMANY/IMI/VALUE", "GERMANY IMI VALUE", "664271"),
    ("GERMANY/IMI/GROWTH", "GERMANY IMI GROWTH", "664376"),
    ("IRELAND/STANDARD/BLEND", "IRELAND", "937200"),
    ("IRELAND/STANDARD/VALUE", "IRELAND VALUE", "105792"),
    ("IRELAND/STANDARD/GROWTH", "IRELAND GROWTH", "105791"),
    ("IRELAND/SMALL/BLEND", "IRELAND SMALL CAP", "106216"),
    ("IRELAND/SMALL/VALUE", "IRELAND SMALL VALUE", "655115"),
    ("IRELAND/SMALL/GROWTH", "IRELAND SMALL GROWTH", "655220"),
    ("IRELAND/SMID/BLEND", "IRELAND SMID CAP", "661652"),
    ("IRELAND/SMID/VALUE", "IRELAND SMID VALUE", "661757"),
    ("IRELAND/SMID/GROWTH", "IRELAND SMID GROWTH", "661862"),
    ("IRELAND/MID/BLEND", "IRELAND MID CAP", "652539"),
    ("IRELAND/MID/VALUE", "IRELAND MID VALUE", "652644"),
    ("IRELAND/MID/GROWTH", "IRELAND MID GROWTH", "652749"),
    ("IRELAND/IMI/BLEND", "IRELAND IMI", "664169"),
    ("IRELAND/IMI/VALUE", "IRELAND IMI VALUE", "664274"),
    ("IRELAND/IMI/GROWTH", "IRELAND IMI GROWTH", "664379"),
    ("ITALY/STANDARD/BLEND", "ITALY", "938000"),
    ("ITALY/STANDARD/VALUE", "ITALY VALUE", "105794"),
    ("ITALY/STANDARD/GROWTH", "ITALY GROWTH", "105793"),
    ("ITALY/SMALL/BLEND", "ITALY SMALL CAP", "106217"),
    ("ITALY/SMALL/VALUE", "ITALY SMALL VALUE", "655116"),
    ("ITALY/SMALL/GROWTH", "ITALY SMALL GROWTH", "655221"),
    ("ITALY/SMID/BLEND", "ITALY SMID CAP", "661653"),
    ("ITALY/SMID/VALUE", "ITALY SMID VALUE", "661758"),
    ("ITALY/SMID/GROWTH", "ITALY SMID GROWTH", "661863"),
    ("ITALY/MID/BLEND", "ITALY MID CAP", "652540"),
    ("ITALY/MID/VALUE", "ITALY MID VALUE", "652645"),
    ("ITALY/MID/GROWTH", "ITALY MID GROWTH", "652750"),
    ("ITALY/LARGE/BLEND", "ITALY LARGE CAP", "650023"),
    ("ITALY/LARGE/VALUE", "ITALY LARGE VALUE", "650128"),
    ("ITALY/LARGE/GROWTH", "ITALY LARGE GROWTH", "650233"),
    ("ITALY/IMI/BLEND", "ITALY IMI", "664170"),
    ("ITALY/IMI/VALUE", "ITALY IMI VALUE", "664275"),
    ("ITALY/IMI/GROWTH", "ITALY IMI GROWTH", "664380"),
    ("NETHERLANDS/STANDARD/BLEND", "NETHERLANDS", "952800"),
    ("NETHERLANDS/STANDARD/VALUE", "NETHERLANDS VALUE", "105802"),
    ("NETHERLANDS/STANDARD/GROWTH", "NETHERLANDS GROWTH", "105801"),
    ("NETHERLANDS/SMALL/BLEND", "NETHERLANDS SMALL CAP", "106220"),
    ("NETHERLANDS/SMALL/VALUE", "NETHERLANDS SMALL VALUE", "655119"),
    ("NETHERLANDS/SMALL/GROWTH", "NETHERLANDS SMALL GROWTH", "655224"),
    ("NETHERLANDS/SMID/BLEND", "NETHERLANDS SMID CAP", "661656"),
    ("NETHERLANDS/SMID/VALUE", "NETHERLANDS SMID VALUE", "661761"),
    ("NETHERLANDS/SMID/GROWTH", "NETHERLANDS SMID GROWTH", "661866"),
    ("NETHERLANDS/MID/BLEND", "NETHERLANDS MID CAP", "652543"),
    ("NETHERLANDS/MID/VALUE", "NETHERLANDS MID VALUE", "652648"),
    ("NETHERLANDS/MID/GROWTH", "NETHERLANDS MID GROWTH", "652753"),
    ("NETHERLANDS/LARGE/BLEND", "NETHERLANDS LARGE CAP", "650026"),
    ("NETHERLANDS/LARGE/VALUE", "NETHERLANDS LARGE VALUE", "650131"),
    ("NETHERLANDS/LARGE/GROWTH", "NETHERLANDS LARGE GROWTH", "650236"),
    ("NETHERLANDS/IMI/BLEND", "NETHERLANDS IMI", "664173"),
    ("NETHERLANDS/IMI/VALUE", "NETHERLANDS IMI VALUE", "664278"),
    ("NETHERLANDS/IMI/GROWTH", "NETHERLANDS IMI GROWTH", "664383"),
    ("NORWAY/STANDARD/BLEND", "NORWAY", "957800"),
    ("NORWAY/STANDARD/VALUE", "NORWAY VALUE", "105808"),
    ("NORWAY/STANDARD/GROWTH", "NORWAY GROWTH", "105807"),
    ("NORWAY/SMALL/BLEND", "NORWAY SMALL CAP", "106222"),
    ("NORWAY/SMALL/VALUE", "NORWAY SMALL VALUE", "655123"),
    ("NORWAY/SMALL/GROWTH", "NORWAY SMALL GROWTH", "655228"),
    ("NORWAY/SMID/BLEND", "NORWAY SMID CAP", "661660"),
    ("NORWAY/SMID/VALUE", "NORWAY SMID VALUE", "661765"),
    ("NORWAY/SMID/GROWTH", "NORWAY SMID GROWTH", "661870"),
    ("NORWAY/MID/BLEND", "NORWAY MID CAP", "652547"),
    ("NORWAY/MID/VALUE", "NORWAY MID VALUE", "652652"),
    ("NORWAY/MID/GROWTH", "NORWAY MID GROWTH", "652757"),
    ("NORWAY/LARGE/BLEND", "NORWAY LARGE CAP", "650030"),
    ("NORWAY/LARGE/VALUE", "NORWAY LARGE VALUE", "650135"),
    ("NORWAY/IMI/BLEND", "NORWAY IMI", "664177"),
    ("NORWAY/IMI/VALUE", "NORWAY IMI VALUE", "664282"),
    ("NORWAY/IMI/GROWTH", "NORWAY IMI GROWTH", "664387"),
    ("PORTUGAL/STANDARD/BLEND", "PORTUGAL", "962000"),
    ("PORTUGAL/STANDARD/VALUE", "PORTUGAL VALUE", "105941"),
    ("PORTUGAL/STANDARD/GROWTH", "PORTUGAL GROWTH", "105940"),
    ("PORTUGAL/SMALL/BLEND", "PORTUGAL SMALL CAP", "106223"),
    ("PORTUGAL/SMALL/VALUE", "PORTUGAL SMALL VALUE", "655126"),
    ("PORTUGAL/SMALL/GROWTH", "PORTUGAL SMALL GROWTH", "655231"),
    ("PORTUGAL/SMID/BLEND", "PORTUGAL SMID CAP", "661663"),
    ("PORTUGAL/SMID/VALUE", "PORTUGAL SMID VALUE", "661768"),
    ("PORTUGAL/SMID/GROWTH", "PORTUGAL SMID GROWTH", "661873"),
    ("PORTUGAL/MID/BLEND", "PORTUGAL MID CAP", "652550"),
    ("PORTUGAL/MID/GROWTH", "PORTUGAL MID GROWTH", "652760"),
    ("PORTUGAL/LARGE/BLEND", "PORTUGAL LARGE CAP", "650033"),
    ("PORTUGAL/LARGE/VALUE", "PORTUGAL LARGE VALUE", "650138"),
    ("PORTUGAL/LARGE/GROWTH", "PORTUGAL LARGE GROWTH", "650243"),
    ("PORTUGAL/IMI/BLEND", "PORTUGAL IMI", "664180"),
    ("PORTUGAL/IMI/VALUE", "PORTUGAL IMI VALUE", "664285"),
    ("PORTUGAL/IMI/GROWTH", "PORTUGAL IMI GROWTH", "664390"),
    ("SPAIN/STANDARD/BLEND", "SPAIN", "972400"),
    ("SPAIN/STANDARD/VALUE", "SPAIN VALUE", "105814"),
    ("SPAIN/STANDARD/GROWTH", "SPAIN GROWTH", "105813"),
    ("SPAIN/SMALL/BLEND", "SPAIN SMALL CAP", "106225"),
    ("SPAIN/SMALL/VALUE", "SPAIN SMALL VALUE", "655128"),
    ("SPAIN/SMALL/GROWTH", "SPAIN SMALL GROWTH", "655233"),
    ("SPAIN/SMID/BLEND", "SPAIN SMID CAP", "661665"),
    ("SPAIN/SMID/VALUE", "SPAIN SMID VALUE", "661770"),
    ("SPAIN/SMID/GROWTH", "SPAIN SMID GROWTH", "661875"),
    ("SPAIN/MID/BLEND", "SPAIN MID CAP", "652552"),
    ("SPAIN/MID/VALUE", "SPAIN MID VALUE", "652657"),
    ("SPAIN/MID/GROWTH", "SPAIN MID GROWTH", "652762"),
    ("SPAIN/LARGE/BLEND", "SPAIN LARGE CAP", "650035"),
    ("SPAIN/LARGE/VALUE", "SPAIN LARGE VALUE", "650140"),
    ("SPAIN/LARGE/GROWTH", "SPAIN LARGE GROWTH", "650245"),
    ("SPAIN/IMI/BLEND", "SPAIN IMI", "664182"),
    ("SPAIN/IMI/VALUE", "SPAIN IMI VALUE", "664287"),
    ("SPAIN/IMI/GROWTH", "SPAIN IMI GROWTH", "664392"),
    ("SWEDEN/STANDARD/BLEND", "SWEDEN", "975200"),
    ("SWEDEN/STANDARD/VALUE", "SWEDEN VALUE", "105818"),
    ("SWEDEN/STANDARD/GROWTH", "SWEDEN GROWTH", "105817"),
    ("SWEDEN/SMALL/BLEND", "SWEDEN SMALL CAP", "106226"),
    ("SWEDEN/SMALL/VALUE", "SWEDEN SMALL VALUE", "655129"),
    ("SWEDEN/SMALL/GROWTH", "SWEDEN SMALL GROWTH", "655234"),
    ("SWEDEN/SMID/BLEND", "SWEDEN SMID CAP", "661666"),
    ("SWEDEN/SMID/VALUE", "SWEDEN SMID VALUE", "661771"),
    ("SWEDEN/SMID/GROWTH", "SWEDEN SMID GROWTH", "661876"),
    ("SWEDEN/MID/BLEND", "SWEDEN MID CAP", "652553"),
    ("SWEDEN/MID/VALUE", "SWEDEN MID VALUE", "652658"),
    ("SWEDEN/MID/GROWTH", "SWEDEN MID GROWTH", "652763"),
    ("SWEDEN/LARGE/BLEND", "SWEDEN LARGE CAP", "650036"),
    ("SWEDEN/LARGE/VALUE", "SWEDEN LARGE VALUE", "650141"),
    ("SWEDEN/LARGE/GROWTH", "SWEDEN LARGE GROWTH", "650246"),
    ("SWEDEN/IMI/BLEND", "SWEDEN IMI", "664183"),
    ("SWEDEN/IMI/VALUE", "SWEDEN IMI VALUE", "664288"),
    ("SWEDEN/IMI/GROWTH", "SWEDEN IMI GROWTH", "664393"),
    ("SWITZERLAND/STANDARD/BLEND", "SWITZERLAND", "975600"),
    ("SWITZERLAND/STANDARD/VALUE", "SWITZERLAND VALUE", "105822"),
    ("SWITZERLAND/STANDARD/GROWTH", "SWITZERLAND GROWTH", "105821"),
    ("SWITZERLAND/SMALL/BLEND", "SWITZERLAND SMALL CAP", "106227"),
    ("SWITZERLAND/SMALL/VALUE", "SWITZERLAND SMALL VALUE", "655130"),
    ("SWITZERLAND/SMALL/GROWTH", "SWITZERLAND SMALL GROWTH", "655235"),
    ("SWITZERLAND/SMID/BLEND", "SWITZERLAND SMID CAP", "661667"),
    ("SWITZERLAND/SMID/VALUE", "SWITZERLAND SMID VALUE", "661772"),
    ("SWITZERLAND/SMID/GROWTH", "SWITZERLAND SMID GROWTH", "661877"),
    ("SWITZERLAND/MID/BLEND", "SWITZERLAND MID CAP", "652554"),
    ("SWITZERLAND/MID/VALUE", "SWITZERLAND MID VALUE", "652659"),
    ("SWITZERLAND/MID/GROWTH", "SWITZERLAND MID GROWTH", "652764"),
    ("SWITZERLAND/LARGE/BLEND", "SWITZERLAND LARGE CAP", "650037"),
    ("SWITZERLAND/LARGE/VALUE", "SWITZERLAND LARGE VALUE", "650142"),
    ("SWITZERLAND/LARGE/GROWTH", "SWITZERLAND LARGE GROWTH", "650247"),
    ("SWITZERLAND/IMI/BLEND", "SWITZERLAND IMI", "664184"),
    ("SWITZERLAND/IMI/VALUE", "SWITZERLAND IMI VALUE", "664289"),
    ("SWITZERLAND/IMI/GROWTH", "SWITZERLAND IMI GROWTH", "664394"),
    ("UNITED KINGDOM/STANDARD/BLEND", "UNITED KINGDOM", "982600"),
    ("UNITED KINGDOM/STANDARD/VALUE", "UNITED KINGDOM VALUE", "105824"),
    ("UNITED KINGDOM/STANDARD/GROWTH", "UNITED KINGDOM GROWTH", "105823"),
    ("UNITED KINGDOM/SMALL/BLEND", "UNITED KINGDOM SMALL CAP", "106228"),
    ("UNITED KINGDOM/SMALL/VALUE", "UNITED KINGDOM SMALL VALUE", "655132"),
    ("UNITED KINGDOM/SMALL/GROWTH", "UNITED KINGDOM SMALL GROWTH", "655237"),
    ("UNITED KINGDOM/SMID/BLEND", "UNITED KINGDOM SMID CAP", "661669"),
    ("UNITED KINGDOM/SMID/VALUE", "UNITED KINGDOM SMID VALUE", "661774"),
    ("UNITED KINGDOM/SMID/GROWTH", "UNITED KINGDOM SMID GROWTH", "661879"),
    ("UNITED KINGDOM/MID/BLEND", "UNITED KINGDOM MID CAP", "652556"),
    ("UNITED KINGDOM/MID/VALUE", "UNITED KINGDOM MID VALUE", "652661"),
    ("UNITED KINGDOM/MID/GROWTH", "UNITED KINGDOM MID GROWTH", "652766"),
    ("UNITED KINGDOM/LARGE/BLEND", "UNITED KINGDOM LARGE CAP", "650039"),
    ("UNITED KINGDOM/LARGE/VALUE", "UNITED KINGDOM LARGE VALUE", "650144"),
    ("UNITED KINGDOM/LARGE/GROWTH", "UNITED KINGDOM LARGE GROWTH", "650249"),
    ("UNITED KINGDOM/IMI/BLEND", "UNITED KINGDOM IMI", "664186"),
    ("UNITED KINGDOM/IMI/VALUE", "UNITED KINGDOM IMI VALUE", "664291"),
    ("UNITED KINGDOM/IMI/GROWTH", "UNITED KINGDOM IMI GROWTH", "664396"),
    ("AUSTRALIA/STANDARD/BLEND", "AUSTRALIA", "903600"),
    ("AUSTRALIA/STANDARD/VALUE", "AUSTRALIA VALUE", "105772"),
    ("AUSTRALIA/STANDARD/GROWTH", "AUSTRALIA GROWTH", "105771"),
    ("AUSTRALIA/SMALL/BLEND", "AUSTRALIA SMALL CAP", "106207"),
    ("AUSTRALIA/SMALL/VALUE", "AUSTRALIA SMALL VALUE", "655093"),
    ("AUSTRALIA/SMALL/GROWTH", "AUSTRALIA SMALL GROWTH", "655198"),
    ("AUSTRALIA/SMID/BLEND", "AUSTRALIA SMID CAP", "661630"),
    ("AUSTRALIA/SMID/VALUE", "AUSTRALIA SMID VALUE", "661735"),
    ("AUSTRALIA/SMID/GROWTH", "AUSTRALIA SMID GROWTH", "661840"),
    ("AUSTRALIA/MID/BLEND", "AUSTRALIA MID CAP", "652517"),
    ("AUSTRALIA/MID/VALUE", "AUSTRALIA MID VALUE", "652622"),
    ("AUSTRALIA/MID/GROWTH", "AUSTRALIA MID GROWTH", "652727"),
    ("AUSTRALIA/LARGE/BLEND", "AUSTRALIA LARGE CAP", "650000"),
    ("AUSTRALIA/LARGE/VALUE", "AUSTRALIA LARGE VALUE", "650105"),
    ("AUSTRALIA/LARGE/GROWTH", "AUSTRALIA LARGE GROWTH", "650210"),
    ("AUSTRALIA/IMI/BLEND", "AUSTRALIA IMI", "664147"),
    ("AUSTRALIA/IMI/VALUE", "AUSTRALIA IMI VALUE", "664252"),
    ("AUSTRALIA/IMI/GROWTH", "AUSTRALIA IMI GROWTH", "664357"),
    ("HONG KONG/STANDARD/BLEND", "HONG KONG", "934400"),
    ("HONG KONG/STANDARD/VALUE", "HONG KONG VALUE", "105790"),
    ("HONG KONG/STANDARD/GROWTH", "HONG KONG GROWTH", "105789"),
    ("HONG KONG/SMALL/BLEND", "HONG KONG SMALL CAP", "106215"),
    ("HONG KONG/SMALL/VALUE", "HONG KONG SMALL VALUE", "655114"),
    ("HONG KONG/SMALL/GROWTH", "HONG KONG SMALL GROWTH", "655219"),
    ("HONG KONG/SMID/BLEND", "HONG KONG SMID CAP", "661651"),
    ("HONG KONG/SMID/VALUE", "HONG KONG SMID VALUE", "661756"),
    ("HONG KONG/SMID/GROWTH", "HONG KONG SMID GROWTH", "661861"),
    ("HONG KONG/MID/BLEND", "HONG KONG MID CAP", "652538"),
    ("HONG KONG/MID/VALUE", "HONG KONG MID VALUE", "652643"),
    ("HONG KONG/MID/GROWTH", "HONG KONG MID GROWTH", "652748"),
    ("HONG KONG/LARGE/BLEND", "HONG KONG LARGE CAP", "650021"),
    ("HONG KONG/LARGE/VALUE", "HONG KONG LARGE VALUE", "650126"),
    ("HONG KONG/LARGE/GROWTH", "HONG KONG LARGE GROWTH", "650231"),
    ("HONG KONG/IMI/BLEND", "HONG KONG IMI", "664168"),
    ("HONG KONG/IMI/VALUE", "HONG KONG IMI VALUE", "664273"),
    ("HONG KONG/IMI/GROWTH", "HONG KONG IMI GROWTH", "664378"),
    ("JAPAN/STANDARD/BLEND", "JAPAN", "939200"),
    ("JAPAN/STANDARD/VALUE", "JAPAN VALUE", "105796"),
    ("JAPAN/STANDARD/GROWTH", "JAPAN GROWTH", "105795"),
    ("JAPAN/SMALL/BLEND", "JAPAN SMALL CAP", "106218"),
    ("JAPAN/SMALL/VALUE", "JAPAN SMALL VALUE", "655117"),
    ("JAPAN/SMALL/GROWTH", "JAPAN SMALL GROWTH", "655222"),
    ("JAPAN/SMID/BLEND", "JAPAN SMID CAP", "661654"),
    ("JAPAN/SMID/VALUE", "JAPAN SMID VALUE", "661759"),
    ("JAPAN/SMID/GROWTH", "JAPAN SMID GROWTH", "661864"),
    ("JAPAN/MID/BLEND", "JAPAN MID CAP", "652541"),
    ("JAPAN/MID/VALUE", "JAPAN MID VALUE", "652646"),
    ("JAPAN/MID/GROWTH", "JAPAN MID GROWTH", "652751"),
    ("JAPAN/LARGE/BLEND", "JAPAN LARGE CAP", "650024"),
    ("JAPAN/LARGE/VALUE", "JAPAN LARGE VALUE", "650129"),
    ("JAPAN/LARGE/GROWTH", "JAPAN LARGE GROWTH", "650234"),
    ("JAPAN/IMI/BLEND", "JAPAN IMI", "664171"),
    ("JAPAN/IMI/VALUE", "JAPAN IMI VALUE", "664276"),
    ("JAPAN/IMI/GROWTH", "JAPAN IMI GROWTH", "664381"),
    ("NEW ZEALAND/STANDARD/BLEND", "NEW ZEALAND", "955400"),
    ("NEW ZEALAND/STANDARD/VALUE", "NEW ZEALAND VALUE", "105804"),
    ("NEW ZEALAND/STANDARD/GROWTH", "NEW ZEALAND GROWTH", "105803"),
    ("NEW ZEALAND/SMALL/BLEND", "NEW ZEALAND SMALL CAP", "106221"),
    ("NEW ZEALAND/SMALL/VALUE", "NEW ZEALAND SMALL VALUE", "655120"),
    ("NEW ZEALAND/SMALL/GROWTH", "NEW ZEALAND SMALL GROWTH", "655225"),
    ("NEW ZEALAND/SMID/BLEND", "NEW ZEALAND SMID CAP", "661657"),
    ("NEW ZEALAND/SMID/VALUE", "NEW ZEALAND SMID VALUE", "661762"),
    ("NEW ZEALAND/SMID/GROWTH", "NEW ZEALAND SMID GROWTH", "661867"),
    ("NEW ZEALAND/MID/BLEND", "NEW ZEALAND MID CAP", "652544"),
    ("NEW ZEALAND/MID/VALUE", "NEW ZEALAND MID VALUE", "652649"),
    ("NEW ZEALAND/MID/GROWTH", "NEW ZEALAND MID GROWTH", "652754"),
    ("NEW ZEALAND/IMI/BLEND", "NEW ZEALAND IMI", "664174"),
    ("NEW ZEALAND/IMI/VALUE", "NEW ZEALAND IMI VALUE", "664279"),
    ("NEW ZEALAND/IMI/GROWTH", "NEW ZEALAND IMI GROWTH", "664384"),
    ("NIHONKABU/STANDARD/BLEND", "NIHONKABU", "128680"),
    ("NIHONKABU/IMI/BLEND", "NIHONKABU IMI", "704167"),
    ("SINGAPORE/STANDARD/BLEND", "SINGAPORE", "998100"),
    ("SINGAPORE/STANDARD/VALUE", "SINGAPORE VALUE", "105812"),
    ("SINGAPORE/STANDARD/GROWTH", "SINGAPORE GROWTH", "105811"),
    ("SINGAPORE/SMALL/BLEND", "SINGAPORE SMALL CAP", "106224"),
    ("SINGAPORE/SMALL/VALUE", "SINGAPORE SMALL VALUE", "655127"),
    ("SINGAPORE/SMALL/GROWTH", "SINGAPORE SMALL GROWTH", "655232"),
    ("SINGAPORE/SMID/BLEND", "SINGAPORE SMID CAP", "661664"),
    ("SINGAPORE/SMID/VALUE", "SINGAPORE SMID VALUE", "661769"),
    ("SINGAPORE/SMID/GROWTH", "SINGAPORE SMID GROWTH", "661874"),
    ("SINGAPORE/MID/BLEND", "SINGAPORE MID CAP", "652551"),
    ("SINGAPORE/MID/VALUE", "SINGAPORE MID VALUE", "652656"),
    ("SINGAPORE/MID/GROWTH", "SINGAPORE MID GROWTH", "652761"),
    ("SINGAPORE/LARGE/BLEND", "SINGAPORE LARGE CAP", "650034"),
    ("SINGAPORE/LARGE/VALUE", "SINGAPORE LARGE VALUE", "650139"),
    ("SINGAPORE/LARGE/GROWTH", "SINGAPORE LARGE GROWTH", "650244"),
    ("SINGAPORE/IMI/BLEND", "SINGAPORE IMI", "664181"),
    ("SINGAPORE/IMI/VALUE", "SINGAPORE IMI VALUE", "664286"),
    ("SINGAPORE/IMI/GROWTH", "SINGAPORE IMI GROWTH", "664391"),
    ("CANADA/STANDARD/BLEND", "CANADA", "912400"),
    ("CANADA/STANDARD/VALUE", "CANADA VALUE", "105778"),
    ("CANADA/STANDARD/GROWTH", "CANADA GROWTH", "105777"),
    ("CANADA/SMALL/BLEND", "CANADA SMALL CAP", "106210"),
    ("CANADA/SMALL/VALUE", "CANADA SMALL VALUE", "655096"),
    ("CANADA/SMALL/GROWTH", "CANADA SMALL GROWTH", "655201"),
    ("CANADA/SMID/BLEND", "CANADA SMID CAP", "661633"),
    ("CANADA/SMID/VALUE", "CANADA SMID VALUE", "661738"),
    ("CANADA/SMID/GROWTH", "CANADA SMID GROWTH", "661843"),
    ("CANADA/MID/BLEND", "CANADA MID CAP", "652520"),
    ("CANADA/MID/VALUE", "CANADA MID VALUE", "652625"),
    ("CANADA/MID/GROWTH", "CANADA MID GROWTH", "652730"),
    ("CANADA/LARGE/BLEND", "CANADA LARGE CAP", "650003"),
    ("CANADA/LARGE/VALUE", "CANADA LARGE VALUE", "650108"),
    ("CANADA/LARGE/GROWTH", "CANADA LARGE GROWTH", "650213"),
    ("CANADA/IMI/BLEND", "CANADA IMI", "664150"),
    ("CANADA/IMI/VALUE", "CANADA IMI VALUE", "664255"),
    ("CANADA/IMI/GROWTH", "CANADA IMI GROWTH", "664360"),
    ("USA/STANDARD/BLEND", "USA", "984000"),
    ("USA/STANDARD/VALUE", "USA VALUE", "105826"),
    ("USA/STANDARD/GROWTH", "USA GROWTH", "105825"),
    ("USA/SMALL/BLEND", "USA SMALL CAP", "106229"),
    ("USA/SMALL/VALUE", "USA SMALL VALUE", "655133"),
    ("USA/SMALL/GROWTH", "USA SMALL GROWTH", "655238"),
    ("USA/SMID/BLEND", "USA SMID CAP", "661670"),
    ("USA/SMID/VALUE", "USA SMID VALUE", "661775"),
    ("USA/SMID/GROWTH", "USA SMID GROWTH", "661880"),
    ("USA/MID/BLEND", "USA MID CAP", "652557"),
    ("USA/MID/VALUE", "USA MID VALUE", "652662"),
    ("USA/MID/GROWTH", "USA MID GROWTH", "652767"),
    ("USA/LARGE/BLEND", "USA LARGE CAP", "650040"),
    ("USA/LARGE/VALUE", "USA LARGE VALUE", "650145"),
    ("USA/LARGE/GROWTH", "USA LARGE GROWTH", "650250"),
    ("USA/IMI/BLEND", "USA IMI", "664187"),
    ("USA/IMI/VALUE", "USA IMI VALUE", "664292"),
    ("USA/IMI/GROWTH", "USA IMI GROWTH", "664397"),
    ("ISRAEL/STANDARD/BLEND", "ISRAEL", "300400"),
    ("ISRAEL/STANDARD/VALUE", "ISRAEL VALUE", "105913"),
    ("ISRAEL/STANDARD/GROWTH", "ISRAEL GROWTH", "105912"),
    ("ISRAEL/SMALL/BLEND", "ISRAEL SMALL CAP", "655074"),
    ("ISRAEL/SMALL/VALUE", "ISRAEL SMALL VALUE", "655179"),
    ("ISRAEL/SMALL/GROWTH", "ISRAEL SMALL GROWTH", "655284"),
    ("ISRAEL/SMID/BLEND", "ISRAEL SMID CAP", "661716"),
    ("ISRAEL/SMID/VALUE", "ISRAEL SMID VALUE", "661821"),
    ("ISRAEL/SMID/GROWTH", "ISRAEL SMID GROWTH", "661926"),
    ("ISRAEL/MID/BLEND", "ISRAEL MID CAP", "652603"),
    ("ISRAEL/MID/VALUE", "ISRAEL MID VALUE", "652708"),
    ("ISRAEL/MID/GROWTH", "ISRAEL MID GROWTH", "652813"),
    ("ISRAEL/LARGE/BLEND", "ISRAEL LARGE CAP", "650086"),
    ("ISRAEL/LARGE/GROWTH", "ISRAEL LARGE GROWTH", "650296"),
    ("ISRAEL/IMI/BLEND", "ISRAEL IMI", "664233"),
    ("ISRAEL/IMI/VALUE", "ISRAEL IMI VALUE", "664338"),
    ("ISRAEL/IMI/GROWTH", "ISRAEL IMI GROWTH", "664443"),
)

In [6]:
index_variants = {"GRTR": "Gross", "NETR": "Net"}

In [ ]:
batched_indexes = [
    (
        batched_index_path,
        batched_index_name,
        batched_index_code,
        index_variant,
        tax_treatment,
    )
    for batched_index_path, batched_index_name, batched_index_code in [
        [
            [index[0] for index in indexes],
            [index[1] for index in indexes],
            [index[2] for index in indexes],
        ]
        for indexes in batched(indexes, 6)
    ]
    for index_variant, tax_treatment in index_variants.items()
]

In [ ]:
for (
    batched_index_path,
    batched_index_name,
    batched_index_code,
    index_variant,
    tax_treatment,
) in batched_indexes:
    async with httpx.AsyncClient() as client:
        tasks = (
            client.get(
                "https://app2-nv.msci.com/products/service/index/indexmaster/downloadLevelData",
                params={
                    "output": "INDEX_LEVELS",
                    "currency_symbol": "USD",
                    "index_variant": index_variant,
                    "start_date": start_date,
                    "end_date": end_date,
                    "data_frequency": "END_OF_MONTH",
                    "baseValue": "false",
                    "index_codes": ",".join(batched_index_code),
                },
                timeout=20,
            )
            for start_date, end_date in (
                ("19691231", "20001130"),
                (
                    "20001229",
                    (
                        pd.Timestamp.now() + pd.tseries.offsets.BMonthEnd(-1, True)
                    ).strftime("%Y%m%d"),
                ),
            )
        )
        responses = await asyncio.gather(*tasks)
        df = pd.concat([read_msci_excel(response.content) for response in responses])

        for index_path, index_name, index_code, column_name in zip(
            batched_index_path,
            batched_index_name,
            batched_index_code,
            df.columns,
            strict=True,
        ):
            os.makedirs(
                os.path.join(os.getcwd(), "test", "MSCI", *index_path.split("/")),
                exist_ok=True,
            )
            df[column_name].dropna().to_csv(
                os.path.join(
                    os.getcwd(),
                    "data",
                    "MSCI",
                    *index_path.split("/"),
                    f"MSCI {index_name} USD {tax_treatment} Monthly.csv",
                )
            )

In [ ]:
for (
    batched_index_path,
    batched_index_name,
    batched_index_code,
    index_variant,
    tax_treatment,
) in batched_indexes:
    async with httpx.AsyncClient() as client:
        tasks = (
            client.get(
                "https://app2-nv.msci.com/products/service/index/indexmaster/downloadLevelData",
                params={
                    "output": "INDEX_LEVELS",
                    "currency_symbol": "USD",
                    "index_variant": index_variant,
                    "start_date": start_date,
                    "end_date": end_date,
                    "data_frequency": "DAILY",
                    "baseValue": "false",
                    "index_codes": ",".join(batched_index_code),
                },
                timeout=20,
            )
            for start_date, end_date in zip(
                pd.date_range("2001-01-01", "today", freq="5BYS-JAN").strftime(
                    "%Y%m%d"
                ),
                pd.date_range("2005-01-01", "today", freq="5BYE-DEC")
                .union(
                    [pd.to_datetime("today") + pd.tseries.offsets.BMonthEnd(-1, True)]
                )
                .strftime("%Y%m%d"),
            )
        )
        responses = await asyncio.gather(*tasks)
        df = pd.concat([read_msci_excel(response.content) for response in responses])

        for index_path, index_name, index_code, column_name in zip(
            batched_index_path,
            batched_index_name,
            batched_index_code,
            df.columns,
            strict=True,
        ):
            os.makedirs(
                os.path.join(os.getcwd(), "test", "MSCI", *index_path.split("/")),
                exist_ok=True,
            )
            df[column_name].dropna().to_csv(
                os.path.join(
                    os.getcwd(),
                    "data",
                    "MSCI",
                    *index_path.split("/"),
                    f"MSCI {index_name} USD {tax_treatment} Daily.csv",
                )
            )